In [53]:
import pandas as pd
import numpy as np


In [54]:
# func 모음

# 1. 종목코드를 6자리로 통일. ex) 990 -> 000990
def padding(x):
    x = str(x)
    return x.rjust(6, "0")

# 2. 결산월 12월인 것만 선택
# : 전체 데이터 4963개 중에서 12월 결산인 데이터가 4836개이다. 나머지 결산월의 데이터가 전체의 2.5%로 데이터량이 작기도하고 결측치가 많아 제거
def func_yearin(x):
    if x[5:7] == '12':
        return True
    else:
        return False

# 3. 회계년도컬럼, 년도만 뽑아서 int화
def change_year(x):
    return int(x[:4])

# 4. 연결, 개별재무제표 통합을 위해 연결재무 데이터 컬럼명 통일화 : "(IFRS연결)"제거
def func_replace_col1(x):
    x = x.replace(" ","")
    x = x.replace(",","")
    x = x.replace("(비교)","")
    x = x.replace("(IFRS)","")
    x = x.replace("(*)","")
    x = x.replace('(연결)',"")
    x = x.replace("[제조]","")
    return x.replace("(IFRS연결)","")

# 5. 연결, 개별재무제표 통합을 위해 개별재무 데이터 컬럼명 통일화 : "(IFRS)"제거
def func_replace_col2(x):
    x = x.replace(" ","")
    x = x.replace(",","")
    x = x.replace("(비교)","")
    x = x.replace("(IFRS)","")
    x = x.replace("(*)","")
    x = x.replace("[제조]","")
    return x.replace("(IFRS)","")

# 6. 2020년 raw data 제외
def func_rm_2020(x):
    if x[:4] == "2020":
        return False
    else:
        return True

# 7. True, False 반전
def func_not(x):
    return not x

In [55]:
개별RDS = pd.read_csv('./datasets/raw/개별RDS.csv',encoding='cp949')
연결RDS = pd.read_csv('./datasets/raw/연결RDS.csv',encoding='cp949')


In [56]:
연결RDS["거래소코드"] = 연결RDS["거래소코드"].map(padding) # 거래소코드 padding
연결RDS["회계년도"] = 연결RDS["회계년도"].map(change_year) # 회계년도 int 로 바꿈

col =[]
for i in 연결RDS.columns:
    col.append(func_replace_col1(i))
연결RDS.columns = col

In [57]:
## 중복된 값 제거
연결RDS.drop_duplicates(['거래소코드',"회계년도"], keep='last', inplace=True)
연결RDS.reset_index(drop=True, inplace=True)

## 외국계기업제거
def func_isnt_fcompany(x):
    if x[:1] == '9':
        return False
    else:
        return True

연결RDS = 연결RDS[연결RDS["거래소코드"].map(func_isnt_fcompany)]

# 중복 거래소코드 제거
def func_overlap_code(x):
    if x == (연결RDS[연결RDS["회사명"]==company_name]["거래소코드"].iloc[-1]) :
        return True
    else:
        return False

list_companyname = list(연결RDS["회사명"].unique())
dfs = list()


for i in list_companyname:
    company_name = i
    #각 기업별로 데이터프레임 생성
    df_concat = 연결RDS[연결RDS["회사명"]==i]
    df_concat = df_concat[df_concat["거래소코드"].map(func_overlap_code)] # 거래소코드 바뀌기 전의 거래소코드의 데이터는 삭제
    dfs.append(df_concat)
연결RDS = pd.concat(dfs)
연결RDS.reset_index(drop=True, inplace=True)


## 기업명 변경
연결RDS[연결RDS['거래소코드']=='036420']
연결RDS.loc[연결RDS['회사명']=='(주)제이콘텐트리',['회사명']] = '(주)콘텐트리중앙'

###################################################################################################################################################################################

In [58]:
개별RDS["거래소코드"] = 개별RDS["거래소코드"].map(padding) # 거래소코드 padding
개별RDS["회계년도"] = 개별RDS["회계년도"].map(change_year) # 회계년도 int로 바꿈

col =[]
for i in 개별RDS.columns:
    col.append(func_replace_col2(i))
개별RDS.columns = col

In [59]:
## 중복된 값 제거
개별RDS.drop_duplicates(['거래소코드',"회계년도"], keep='last', inplace=True)
개별RDS.reset_index(drop=True, inplace=True)

## 외국계기업제거
def func_isnt_fcompany(x):
    if x[:1] == '9':
        return False
    else:
        return True

개별RDS = 개별RDS[개별RDS["거래소코드"].map(func_isnt_fcompany)]

# 중복 거래소코드 제거
def func_overlap_code(x):
    if x == (개별RDS[개별RDS["회사명"]==company_name]["거래소코드"].iloc[-1]) :
        return True
    else:
        return False

list_companyname = list(개별RDS["회사명"].unique())
dfs = list()


for i in list_companyname:
    company_name = i
    #각 기업별로 데이터프레임 생성
    df_concat = 개별RDS[개별RDS["회사명"]==i]
    df_concat = df_concat[df_concat["거래소코드"].map(func_overlap_code)] # 거래소코드 바뀌기 전의 거래소코드의 데이터는 삭제
    dfs.append(df_concat)
개별RDS = pd.concat(dfs)
개별RDS.reset_index(drop=True, inplace=True)


## 기업명 변경
개별RDS[개별RDS['거래소코드']=='036420']
개별RDS.loc[개별RDS['회사명']=='(주)제이콘텐트리',['회사명']] = '(주)콘텐트리중앙'

In [60]:
개별RDS.isna().sum()

회사명            0
거래소코드          0
회계년도           0
연구비          241
경상개발비        241
매출액(영업수익)    241
dtype: int64

In [61]:
연결RDS.isna().sum()

회사명             0
거래소코드           0
회계년도            0
연구비          4766
경상개발비        4766
매출액(영업수익)     241
dtype: int64

In [62]:
# df_재무를 완전데이터와 결측데이터로 나눈 뒤 결측데이터를 개별재무데이터로 대체시키고, df_재무로 다시 통합하면 끝!
RDS완전 =연결RDS[연결RDS["연구비"].isna().map(func_not)]
# df_재무_완전

RDS결측 =연결RDS[연결RDS["연구비"].isna()]
# df_재무_결측

list_na_col = ['연구비','경상개발비']

RDS결측 = RDS결측.drop(list_na_col,axis=1)
RDS결측 = pd.merge(RDS결측,개별RDS[['거래소코드','회계년도']+list_na_col],on=['거래소코드','회계년도'],how='left')

RDS_df  = pd.concat([RDS완전,RDS결측],axis=0)

In [68]:
print( '결측치가 존재하는 기업의 수',len(RDS_df[RDS_df.isna().any(1)].회사명.unique()),'개')

결측치가 존재하는 기업의 수 126 개


In [69]:
## 결측치 제거
RDS_df = RDS_df[RDS_df["연구비"].isna().map(func_not)] # 위 결측치 없는 데이터로 필터링
RDS_df

RDS_df.isna().sum().to_dict()

{'회사명': 0, '거래소코드': 0, '회계년도': 0, '연구비': 0, '경상개발비': 0, '매출액(영업수익)': 0}

####################################################################################################################################################################################

In [72]:
RDS_df.to_csv('RDS.csv',encoding='UTF-8-sig')